# Saving traces

In [1]:
from reader import Reader
import matplotlib.pyplot as plt
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from datetime import datetime
from datetime import date as date_creator
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from numba import njit

%matplotlib inline

from os import listdir
from os.path import isfile, join
import shutil
import os

In [2]:
def smooth(x,window_len=11,window='hanning'):
        if x.ndim != 1:
                raise ValueError("smooth only accepts 1 dimension arrays.")
        if x.size < window_len:
                raise ValueError("Input vector needs to be bigger than window size.")
        if window_len<3:
                return x
        if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
                raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
        s=np.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
        if window == 'flat': #moving average
                w=np.ones(window_len,'d')
        else:
                w=eval('np.'+window+'(window_len)')
        y=np.convolve(w/w.sum(),s,mode='same')
        return y[window_len:-window_len+1]
def get_date_time_dif(start_time,stop_time):
    date = date_creator(1, 1, 1)
    datetime1 = datetime.combine(date, start_time)
    datetime2 = datetime.combine(date, stop_time)
    time_elapsed = datetime1 - datetime2
    return time_elapsed.total_seconds()
def compute_dist(lat1,lon1,lat2,lon2,rad=True):
    if not(rad):
        lat1 = radians(lat1)
        lon1 = radians(lon1)
        lat2 = radians(lat2)
        lon2 = radians(lon2)
    # approximate radius of earth in m
    R = 6373_000.0
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c
def remove_zero_from_alti(alti):
    if alti[0] < 10 :
        alti[0] = alti[1]
    if alti[-1] < 10 :
        alti[-1] = alti[-2]
    for i,alt in enumerate(alti):
        if alt < 10 :
            if i >= len(alti)-1 :
                alti[i] = (alti[i-1]+alti[i])/2
            else :  
                alti[i] = (alti[i-1]+alti[i+1])/2
    return alti

# def read_igc(file_url):
#     igc = requests.get(file_url)
#     parsed_igc_file = Reader().read(io.StringIO(igc.text))
def read_igc(file):
    with open(file, 'r') as f:
        parsed_igc_file = Reader().read(f)
    # print('igc_file created')
    previous_lat = 0
    previous_lon = 0
    
    all_lon = np.zeros(len(parsed_igc_file['fix_records'][1]))
    all_lat = np.zeros(len(parsed_igc_file['fix_records'][1]))
    all_speed=np.zeros(len(parsed_igc_file['fix_records'][1]))
    all_vz=np.zeros(len(parsed_igc_file['fix_records'][1]))
    all_alti=np.zeros(len(parsed_igc_file['fix_records'][1]))
    all_time=[0 for _ in range(len(parsed_igc_file['fix_records'][1]))]

    for i,record in enumerate(parsed_igc_file['fix_records'][1]):
        record['time'] = record['time'].replace(hour=record['time'].hour + 1)
        all_lat[i] = record['lat']
        all_lon[i] = record['lon']
        if previous_lon ==0 :
            previous_lat = record['lat']
            previous_lon = record['lon']
            previous_datetime = record['time']
            previous_alt_gps = record['gps_alt']
            # previous_alt_baro = record['pressure_alt']
            all_time[i] = record['time']
        else :
            dxy = abs(compute_dist(previous_lat,previous_lon,record['lat'],record['lon'],rad=False))
            # dz = previous_alt_baro-record['pressure_alt']
            dz = previous_alt_gps-record['gps_alt']
            dt = get_date_time_dif(record['time'],previous_datetime)
            # if dt>2 : print('dtttttt>2 ',dt)
            if dt>0 :
                all_speed[i] = min(100,dxy/dt*3.6)
                all_vz[i] = dz/dt
            all_alti[i] = record['gps_alt']
            all_time[i] = record['time']
            
            previous_lat = record['lat']
            previous_lon = record['lon']
            previous_datetime = record['time']
            previous_alt_gps = record['gps_alt']
            # previous_alt_baro = record['pressure_alt']

            # if previous_datetime.hour>10 :
            #     all_speed = all_speed[:i]
            #     all_vz = all_vz[:i]
            #     all_alti = all_alti[:i]
            #     break
    return all_lon,all_lat,all_speed,all_vz,remove_zero_from_alti(all_alti),all_time       

def reshape_array(arr,time_vid):
    nb_img_by_sec = 24
    

    t_true = np.linspace(time_vid[0], time_vid[-1], num=len(time_vid), endpoint=True)
    t_inter = np.linspace(time_vid[0], time_vid[-1], num=int(len(time_vid)*nb_img_by_sec/speed_acc), endpoint=True)
    f = interp1d(t_true, arr, kind='cubic')

    return f(t_inter)

def smooth_igc_output(L_all):
    all_ret = []
    for l_val in L_all:
        l_val[0]=l_val[1]#=np.mean(l_val[:int(len(l_val)/10)])
        smoothed = smooth(l_val,50,'hanning')
        all_ret.append(smoothed)
    return all_ret

def plot_smooth_non_smooth(smooth,non_smooth):
    plt.figure(figsize=(18,9))
    plt.plot(non_smooth)
    plt.plot(smooth)
    plt.show()

def get_last_date_of_all_raw_file(path_raw_file):
    delta_time_writing = 20
    all_ending_time = []
    for file in os.listdir(path_raw_file):
        if "_11_" in file :
            time_end = os.path.getmtime(path_raw_file+'\\'+file)
            all_ending_time.append(datetime.fromtimestamp(time_end-delta_time_writing).time())
    return all_ending_time

def convert_time_to_sec(all_time):
    for i in range(len(all_time)):
        all_time[i] = all_time[i].hour*3600 + all_time[i].minute*60 + all_time[i].second
    return np.array(all_time,dtype=np.float32)

@njit()  
def compute_area(line):
    return np.sum(line[:-1,0]*line[1:,1]-line[1:,0]*line[:-1,1])/2

@njit()  
def get_contour(line):
    contour1 = algo_contour(np.copy(line))
    contour2 = algo_contour(line[::-1])
    if compute_area(contour1)<compute_area(contour2):
        return contour2
    else:
        return contour1

@njit()    
def algo_contour(line):
    i=0
    while i < len(line)-1: # Boucle principale tous les segments
        mina = 1.
        for j in range(i+2,len(line)-1): # Boucle sur les segments suivants pour détecter les intersections
            dxi = line[i+1,0]-line[i,0]
            dxj = line[j+1,0]-line[j,0]
            dyi = line[i+1,1]-line[i,1]
            dyj = line[j+1,1]-line[j,1]
            b = dxi*dyj - dyi*dxj
            if b!=0.:
                a = ( (line[j,0]-line[i,0])*dyj - (line[j,1]-line[i,1])*dxj )/b
                beta = ( (line[j,0]-line[i,0])*dyi - (line[j,1]-line[i,1])*dxi )/b
                if a>0 and a<mina and beta>0 and beta<1:
                    mina = a
                    minj = j
        if mina<1:
            cross = np.array([[ line[i,0]*(1-mina)+line[i+1,0]*mina , line[i,1]*(1-mina)+line[i+1,1]*mina ]])
            line = np.concatenate((line[:i+1],cross,line[minj:i:-1],cross,line[minj+1:]),axis=0)
        else:
            for j in range(1,i):
                if line[i+1,0]==line[j,0] and line[i+1,1]==line[j,1]:
                    dxi = line[i,0]-line[j-1,0]
                    dxj = line[i+2,0]-line[j+1,0]
                    dyi = line[i,1]-line[j-1,1]
                    dyj = line[i+2,1]-line[j+1,1]
                    b = dxi*dyj - dyi*dxj
                    if b<0:
                        line = np.concatenate((line[:j+1],line[i+2:]),axis=0)
                        i = j-1
                        break
        i+=1
    return line

In [3]:
def prep_file(path,file) :
    all_lon,all_lat,all_speed, all_vz, all_alti, all_time = read_igc(join(path,file))
    print('\n process_igc ok')
    
    all_speed_smooth, all_vz_smooth, all_alti_smooth = smooth_igc_output([all_speed, all_vz, all_alti])
    all_vz2 = np.zeros(all_alti_smooth.shape[0])
    
    print('igc smothed')
    for i,alti in enumerate(list(all_alti_smooth)):
      if i >=all_alti_smooth.shape[0]-1 :
          pass
      else :
          all_vz2[i] = all_alti_smooth[i+1]-all_alti_smooth[i]
    
    print('smoothed vario ok')
    speed_vid = all_speed_smooth
    vz_vid = all_vz2
    alti_vid = all_alti_smooth
    time_vid = convert_time_to_sec(all_time)
    print('time to sec ok')

    return all_lon,all_lat,speed_vid,vz_vid,alti_vid,time_vid,all_vz2,all_speed_smooth, all_vz_smooth, all_alti_smooth,all_alti

def liste_files(path):
    return [f for f in listdir(path) if isfile(join(path, f))]

In [4]:
def filter_list_3(lst1, lst2, threshold_low1=None, threshold_high1=None, threshold_low2=None, threshold_high2=None):
    filtered_list1 = []
    filtered_list2 = []
    indexs = []
    for item1,item2 in zip(lst1,lst2):
        if threshold_low1 is not None and item1 < threshold_low1:
            continue
        if threshold_high1 is not None and item1 > threshold_high1:
            continue
        if threshold_low2 is not None and item2 < threshold_low2:
            continue
        if threshold_high2 is not None and item2 > threshold_high2:
            continue
        indexs.append(lst.index(item1))
        filtered_list1.append(item1)
        filtered_list2.append(item2)
    return filtered_list1,filtered_list2,indexs

def missing_numbers(lst):
    if not lst:
        return []

    max_num = max(lst)
    return [num for num in range(1, max_num) if num not in lst]

def remove_elements_by_indices(lst, indices):
    indices = set(indices)  # Convert to set for faster lookup
    return [elem for i, elem in enumerate(lst) if i not in indices]

def save_traces(img_extent,type_vol) :
    directory = 'C:/Users/barroisl/Documents/Stage_Léo/Vols/'
    days = ['IGC_19-08-2023/','IGC_20-08-2023/','IGC_21-08-2023/','IGC_22-08-2023/','IGC_23-08-2023/','IGC_24-08-2023/']
    
    for j in range(len(days)): 
        low_path = join(directory,days[j])
        if type_vol == 'para' :
            path = join(low_path,'Parapente/')
        elif type_vol == 'plan' :
            path = join(low_path,'Planeur/')
        else :
            path = ''
        liste_f = liste_files(path)
        for i,file in enumerate(liste_f):
            print(join(low_path,path))
            print(file)
            print(i,end=' ')
            all_lon,all_lat,speed_vid,vz_vid,alti_vid,time_vid,all_vz2,all_speed_smooth, all_vz_smooth, all_alti_smooth,all_alti = prep_file(join(low_path,path),file)
            lon,lat,indexs = filter_list_3(lon, lat, min_lon-10, max_lon+10,min_lat-10, max_lat+10)
            miss_nbres = missing_numbers(indexs)
            all_speed = remove_elements_by_indices(all_speed, miss_nbres)
            all_alti = remove_elements_by_indices(all_alti, miss_nbres)
            all_vz = remove_elements_by_indices(all_vz, miss_nbres)
            all_time = remove_elements_by_indices(all_time, miss_nbres)
            data, = smooth_igc_output([vz])
            df = {
                'lon': all_lon,
                'lat': all_lat,
                'speed': speed_vid,
                'speed_smooth':all_speed_smooth,
                'vz' : vz_vid,
                'vz2' : all_vz2,
                'vz_smooth': all_vz_smooth,
                'alti' : all_alti,
                'alti smooth' :all_alti_smooth,
                'alti':remove_zero_from_alti(all_alti),
                'time' : time_vid,
                'smooth_spd':data
            }
            df.to_csv(join(join(low_path,path),'all_traces/',join(file,'.txt'), sep='\t'))

In [5]:
img_extent = (4.7942, 8.1545, 43.3545, 46.6707)

save_traces(img_extent,'plan')

C:/Users/barroisl/Documents/Stage_Léo/Vols/IGC_19-08-2023/Planeur/
2023-08-19 Aert Mante 325154.igc
0 

TypeError: Reader.__init__() missing 4 required positional arguments: '_storage', '_search', '_parser', and '_reserved_name_scheme'

In [8]:
file = '2023-08-20-XCT-BLA-01.igc'
path = 'C:/Users/barroisl/Documents/Stage_Léo/Vols/IGC_20-08-2023/Parapente_old/'
all_lon,all_lat,speed_vid,vz_vid,alti_vid,time_vid,all_vz2,all_speed_smooth, all_vz_smooth, all_alti_smooth,all_alti = prep_file(path, file)


TypeError: Reader.__init__() missing 4 required positional arguments: '_storage', '_search', '_parser', and '_reserved_name_scheme'